In [1]:
import torch.nn as nn
import pandas as pd
import torch
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader

device = "cuda"
torch.cuda.is_available()

True

## Loading and Preparing the Data

In [4]:
train_df = pd.read_csv("sign_mnist_train.csv")
valid_df = pd.read_csv("sign_mnist_valid.csv")

In [5]:
sample_df = train_df.head().copy()
sample_df.pop('label')
sample_x = sample_df.values
sample_x

array([[107, 118, 127, ..., 204, 203, 202],
       [155, 157, 156, ..., 103, 135, 149],
       [187, 188, 188, ..., 195, 194, 195],
       [211, 211, 212, ..., 222, 229, 163],
       [164, 167, 170, ..., 163, 164, 179]])

In [6]:
sample_x.shape

(5, 784)

In [7]:
IMG_HEIGHT = 28
IMG_WIDTH = 28
IMG_CHS = 1

sample_x = sample_x.reshape(-1, IMG_CHS, IMG_HEIGHT, IMG_WIDTH)
sample_x.shape

(5, 1, 28, 28)

###  Create a Dataset

In [8]:
class MyDataset(Dataset):
    def __init__(self, base_df):
        x_df = base_df.copy()
        y_df = x_df.pop('label')
        x_df = x_df.values / 255  # Normalize values from 0 to 1
        x_df = x_df.reshape(-1, IMG_CHS, IMG_WIDTH, IMG_HEIGHT)
        self.xs = torch.tensor(x_df).float().to(device)
        self.ys = torch.tensor(y_df).to(device)

    def __getitem__(self, idx):
        x = self.xs[idx]
        y = self.ys[idx]
        return x, y

    def __len__(self):
        return len(self.xs)

### Create a DataLoader

In [9]:
BATCH_SIZE = 32

train_data = MyDataset(train_df)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE,shuffle=True)
train_N = len(train_loader.dataset)

valid_data = MyDataset(valid_df)
valid_loader = DataLoader(valid_data, batch_size=BATCH_SIZE)
valid_N = len(valid_loader.dataset)

In [10]:
batch = next(iter(train_loader))
batch

[tensor([[[[0.7765, 0.7765, 0.7804,  ..., 0.7216, 0.7137, 0.7020],
           [0.7843, 0.7843, 0.7843,  ..., 0.7255, 0.7176, 0.7098],
           [0.7882, 0.7882, 0.7882,  ..., 0.7294, 0.7216, 0.7176],
           ...,
           [0.8745, 0.8784, 0.8824,  ..., 0.8275, 0.8196, 0.8118],
           [0.8745, 0.8824, 0.8863,  ..., 0.8314, 0.8235, 0.8157],
           [0.8745, 0.8824, 0.8863,  ..., 0.8275, 0.8196, 0.8157]]],
 
 
         [[[0.2824, 0.2980, 0.3137,  ..., 0.6980, 0.6941, 0.6824],
           [0.2863, 0.2980, 0.3176,  ..., 0.7098, 0.7059, 0.7020],
           [0.2863, 0.3059, 0.3216,  ..., 0.7255, 0.7255, 0.7216],
           ...,
           [0.4235, 0.4275, 0.4353,  ..., 0.0510, 0.1569, 0.1216],
           [0.4275, 0.4314, 0.4314,  ..., 0.0235, 0.1059, 0.1569],
           [0.4235, 0.4235, 0.4275,  ..., 0.0627, 0.0549, 0.1216]]],
 
 
         [[[0.6078, 0.6196, 0.6314,  ..., 0.6980, 0.7020, 0.6941],
           [0.6118, 0.6275, 0.6431,  ..., 0.7020, 0.6980, 0.6980],
           [0.6157

In [11]:
batch[0].shape

torch.Size([32, 1, 28, 28])

In [12]:
batch[1].shape

torch.Size([32])

## Creating a Convolutional Model

In [13]:
n_classes = 24
kernel_size = 3
flattened_img_size = 75 * 3 * 3

model = nn.Sequential(
    # First convolution
    nn.Conv2d(IMG_CHS, 25, kernel_size, stride=1, padding=1),  # 25 x 28 x 28
    nn.BatchNorm2d(25),
    nn.ReLU(),
    nn.MaxPool2d(2, stride=2),  # 25 x 14 x 14
    # Second convolution
    nn.Conv2d(25, 50, kernel_size, stride=1, padding=1),  # 50 x 14 x 14
    nn.BatchNorm2d(50),
    nn.ReLU(),
    nn.Dropout(.2),
    nn.MaxPool2d(2, stride=2),  # 50 x 7 x 7
    # Third convolution
    nn.Conv2d(50, 75, kernel_size, stride=1, padding=1),  # 75 x 7 x 7
    nn.BatchNorm2d(75),
    nn.ReLU(),
    nn.MaxPool2d(2, stride=2),  # 75 x 3 x 3
    # Flatten to Dense
    nn.Flatten(),
    nn.Linear(flattened_img_size, 512),
    nn.Dropout(.3),
    nn.ReLU(),
    nn.Linear(512, n_classes)
)

In [14]:
model = torch.compile(model.to(device))
model

OptimizedModule(
  (_orig_mod): Sequential(
    (0): Conv2d(1, 25, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(25, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(50, 75, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): BatchNorm2d(75, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (13): Flatten(start_dim=1, end_dim=-1)
    (14): Linear(in_features=675, out_features=512, bias=True)
    (15): Dropout

In [16]:
loss_function = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters())

In [17]:
def get_batch_accuracy(output, y, N):
    pred = output.argmax(dim=1, keepdim=True)
    correct = pred.eq(y.view_as(pred)).sum().item()
    return correct / N

### Training the Model

In [18]:
def validate():
    loss = 0
    accuracy = 0

    model.eval()
    with torch.no_grad():
        for x, y in valid_loader:
            output = model(x)

            loss += loss_function(output, y).item()
            accuracy += get_batch_accuracy(output, y, valid_N)
    print('Valid - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

In [19]:
def train():
    loss = 0
    accuracy = 0

    model.train()
    for x, y in train_loader:
        output = model(x)
        optimizer.zero_grad()
        batch_loss = loss_function(output, y)
        batch_loss.backward()
        optimizer.step()

        loss += batch_loss.item()
        accuracy += get_batch_accuracy(output, y, train_N)
    print('Train - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

In [20]:
epochs = 20

for epoch in range(epochs):
    print('Epoch: {}'.format(epoch))
    train()
    validate()

Epoch: 0
Train - Loss: 280.8173 Accuracy: 0.9020
Valid - Loss: 39.9490 Accuracy: 0.9453
Epoch: 1
Train - Loss: 12.9207 Accuracy: 0.9966
Valid - Loss: 47.1475 Accuracy: 0.9354
Epoch: 2
Train - Loss: 13.8925 Accuracy: 0.9954
Valid - Loss: 15.5456 Accuracy: 0.9760
Epoch: 3
Train - Loss: 12.0218 Accuracy: 0.9960
Valid - Loss: 83.3515 Accuracy: 0.8871
Epoch: 4
Train - Loss: 6.0114 Accuracy: 0.9981
Valid - Loss: 14.4824 Accuracy: 0.9750
Epoch: 5
Train - Loss: 3.2415 Accuracy: 0.9993
Valid - Loss: 48.6344 Accuracy: 0.9387
Epoch: 6
Train - Loss: 14.5428 Accuracy: 0.9950
Valid - Loss: 28.6558 Accuracy: 0.9631
Epoch: 7
Train - Loss: 1.8457 Accuracy: 0.9993
Valid - Loss: 17.2207 Accuracy: 0.9739
Epoch: 8
Train - Loss: 8.4906 Accuracy: 0.9969
Valid - Loss: 24.4311 Accuracy: 0.9723
Epoch: 9
Train - Loss: 7.4465 Accuracy: 0.9977
Valid - Loss: 24.5717 Accuracy: 0.9674
Epoch: 10
Train - Loss: 3.6116 Accuracy: 0.9990
Valid - Loss: 16.7506 Accuracy: 0.9822
Epoch: 11
Train - Loss: 8.4668 Accuracy: 0.9972